<a href="https://colab.research.google.com/github/sandhyaparna/Python-DataScience-CookBook/blob/master/Modeling/SplittingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyfarmhash

     |████████████████████████████████| 102kB 4.2MB/s 
  Created wheel for pyfarmhash: filename=pyfarmhash-0.2.2-cp36-cp36m-linux_x86_64.whl size=66877 sha256=a24cd4a3b158054365208b4091ab60cd177209cc9ebb5d63bf24004e6e96b18c
  Stored in directory: /root/.cache/pip/wheels/3f/41/10/80e03a3ffa9ba29a41eac65c4371ade0bd1e21dbdabb867c31
Successfully built pyfarmhash


In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import io
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving employees.csv to employees.csv


In [ ]:
# specifying which ones are to be considered as missing values
missing_value_formats = ["n.a.","?","NA","n/a", "na", "--"]
Employees = pd.read_csv("employees.csv", na_values = missing_value_formats)

In [ ]:
Employees['Date_of_Birth'] = [np.random.choice(pd.date_range(start='1990-01-01',end='2000-12-31')) for i in range(Employees.shape[0])]

In [ ]:
Employees['Target'] = np.random.randint(0,2, size=Employees.shape[0]) 

In [ ]:
Employees.head()

,First Name,Gender,Salary,Bonus %,Senior Management,Team,Date_of_Birth,Target
0,Douglas,Male,97308.0,6.945,True,Marketing,1997-09-12,0
1,Thomas,Male,61933.0,NaN,True,NaN,1991-03-12,0
2,Maria,Female,130590.0,11.858,False,Finance,1992-12-10,1
3,Jerry,Male,NaN,9.340,True,Finance,1992-01-20,0
4,Larry,Male,101004.0,1.389,True,Client Services,1997-09-14,1


In [ ]:
from sklearn.model_selection import train_test_split

print("\nSplitting 80% project_data into Train and remaining as Test data")
# Seperate Independant variables from Dependent variable
X = Employees.drop(['Target'],axis=1)
y = Employees[['Target']]

# Splitting 80% project_data into Train and remaining as Test data. shuffle=True is default
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=Employees.Target, random_state=42)

# Train & Test data sets
Train = pd.concat([X_train, y_train], axis=1)
Test = pd.concat([X_test, y_test], axis=1)

print(Train.shape)
print(Test.shape)


Splitting 80% project_data into Train and remaining as Test data
(800, 8)
(200, 8)


In [ ]:
pd.value_counts(Employees['First Name']).head()

Marilyn    11
Jeremy     10
Barbara    10
Todd       10
Irene       9
Name: First Name, dtype: int64

In [ ]:
import random

# Split data into Train and Test by First Name
# Unique names are split into 80:20 ratio
Names = pd.unique(Employees['First Name'])
Train_Names = random.sample(list(Names),np.ceil(len(Names)*0.8).astype(int))
Test_Names = set(Names).difference(set(Train_Names))

Train = Employees[Employees['First Name'].isin(Train_Names)]
Test = Employees[Employees['First Name'].isin(Test_Names)]

print(Train.shape)
print(Test.shape)

(823, 8)
(177, 8)


In [ ]:
# Split first 80% rows into Train and remaining 20% rows into Test
Train = Employees.iloc[:int(Employees.shape[0]*0.80)] #Extracting first 80%
Test = Employees.iloc[int(Employees.shape[0]*0.80):] # Exracting remaining

# First 80% of rows as Train and later as Test
Train,Test = np.split(Employees, [int(.8 *len(Employees))])
print(Train.shape)
print(Test.shape)

(800, 8)
(200, 8)


In [ ]:
# Splitting based on date variable
# just filter the data

### FARM_FINGERPRINT 
* Open-source hashing algorithm that is implemented consistently in C++ (and hence: Java or Python) and in BigQuery SQL.
* or example, if you are training a model to predict flight delays, the arrival delays of flights on the same day will be highly correlated with each other. This is called leakage, and it is an important problem to avoid when doing machine learning. All the flights on any given date will belong to the same split — train, valid, or test. This is repeatable regardless of things like the random seed.

In [ ]:
Employees['LoginDate'] = [np.random.choice(pd.date_range(start='2020-06-30',end='2020-07-15')) for i in range(Employees.shape[0])]

In [ ]:
Employees = Employees.dropna()

In [ ]:
# Hash - strings
# Missing values are removed from the column on which we want to apply hash function
import farmhash
Employees['Name_Hash'] = Employees['First Name'].map(farmhash.hash64)
# df.apply(lambda x: farmhash.hash64(x)) # on whole dataframe use apply

In [ ]:
Employees['LoginDate_Hash'] = Employees['LoginDate'].astype(str).map(farmhash.hash64)

In [ ]:
# split data based on hash column
# if remainder <8 then train
Train = Employees[abs(Employees.LoginDate_Hash%10)<8]
Test = Employees[abs(Employees.LoginDate_Hash%10)>=8]
print(Train.shape)
print(Test.shape)

(515, 11)
(227, 11)


In [ ]:
np.unique(abs(Employees.LoginDate_Hash%10))

array([0, 1, 3, 5, 6, 7, 8, 9], dtype=uint64)

In [ ]:
Employees.head()

,First Name,Gender,Salary,Bonus %,Senior Management,Team,Date_of_Birth,Target,LoginDate,Name_Hash,LoginDate_Hash
0,Douglas,Male,97308.0,6.945,True,Marketing,1997-09-12,0,2020-07-04,13089925452571571431,5110218570070884505
2,Maria,Female,130590.0,11.858,False,Finance,1992-12-10,1,2020-07-13,7977770318440479964,12944729872673653367
4,Larry,Male,101004.0,1.389,True,Client Services,1997-09-14,1,2020-07-07,1017167388419051281,16496207191609800238
6,Ruby,Female,65476.0,10.012,True,Product,1993-05-05,0,2020-07-04,6548883630689698183,5110218570070884505
9,Frances,Female,139852.0,7.524,True,Business Development,1998-12-05,1,2020-07-05,4116071532306498785,17670039732204069343


In [ ]:
import farmhash
print(farmhash.hash64('6823339101'))

17557133836170941146


In [ ]:
print(farmhash.fingerprint64('6823339101'))

17557133836170941146


In [ ]:
import hashlib
# hashlib.md5(Test.encode('ascii')).hexdigest()

hashlib.md5(('6823339101').encode()).hexdigest()

'891e2039367d4a8cec11fab477538b8f'

### Cross-validation: manual upsample within folds is same as Using the imblearn pipeline https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html

In [49]:
import warnings
warnings.simplefilter('ignore')

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV
from sklearn.metrics import recall_score

In [50]:
# Extract only numeric variables
Employees_Num = Employees.select_dtypes(include = np.number)

# Seperate Independant variables from Dependent variable
X = Employees_Num.drop(['Target'],axis=1)
y = Employees_Num[['Target']]

# Splitting 80% project_data into Train and remaining as Test data. shuffle=True is default
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=Employees.Target, random_state=42)


In [51]:
Employees_Num.head()

,Salary,Bonus %,Target,Name_Hash,LoginDate_Hash
0,97308.0,6.945,0,13089925452571571431,5110218570070884505
2,130590.0,11.858,1,7977770318440479964,12944729872673653367
4,101004.0,1.389,1,1017167388419051281,16496207191609800238
6,65476.0,10.012,0,6548883630689698183,5110218570070884505
9,139852.0,7.524,1,4116071532306498785,17670039732204069343


In [52]:
imba_pipeline = make_pipeline(SMOTE(random_state=42), 
                              RandomForestClassifier(n_estimators=100, random_state=13))

kf = KFold(n_splits=5, random_state=42, shuffle=False)
cross_val_score(imba_pipeline, X_train, y_train, scoring='recall', cv=kf)
# Returns Array of scores of the estimator for each run of the cross validation.

array([0.57377049, 0.50909091, 0.49315068, 0.51785714, 0.45454545])

In [53]:
# Hyperparameter Tuning
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 10, 12],
    'random_state': [13]
}
new_params = {'randomforestclassifier__' + key: params[key] for key in params}
grid_imba = GridSearchCV(imba_pipeline, param_grid=new_params, cv=kf, scoring='recall', return_train_score=True)
grid_imba.fit(X_train, y_train);

In [54]:
grid_imba.best_params_

{'randomforestclassifier__max_depth': 10,
 'randomforestclassifier__n_estimators': 200,
 'randomforestclassifier__random_state': 13}

In [55]:
grid_imba.best_score_

0.5023917919044799

In [56]:
y_test_predict = grid_imba.predict(X_test)
recall_score(y_test, y_test_predict)

0.3684210526315789